In this notebook we will go over some simple techniques in topic modeling: ```SVD```, ```NMF``` and ```LDA``` . We will also learn about the concepts of ```perplexity``` and ```topic coherence```.

We will be using scikit-learn's ```20newsgroups``` dataset.

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import numpy as np
import nltk
import gensim
from sklearn.datasets import fetch_20newsgroups

## Download and survey the data

In [3]:
# obtain the dataset
# for our purposes it will suffice to only looking at the training set
newsgroups = fetch_20newsgroups(subset = 'train')

In [4]:
# list of newsgroups categories
newsgroups.target_names

['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

For demonstration purposes, let us only consider 7 categories of the newgroups. This will allow us to keep the corpora short and therefore our computing times low. 

In [5]:
categories = ['alt.atheism','comp.graphics','misc.forsale','rec.autos','sci.crypt','soc.religion.christian','talk.politics.guns']
remove = ('headers', 'footers', 'quotes')
newsgroups = fetch_20newsgroups(subset = 'train', categories = categories, remove = remove)

In [15]:
# number of documents in our dataset
newsgroups.filenames.shape

(3983,)

In [23]:
# checkout the first three entries in the dataset
# we will seperate each entry by '\n\n=*=*=*=*=*=*=*=*=\n'
print('\n\n=*=*=*=*=*=*=*=*=\n'.join(newsgroups.data[:3]))



Find an encyclopedia. Volume H. Now look up Hitler, Adolf. He had
many more people than just Germans enamoured with him.

P.

=*=*=*=*=*=*=*=*=


I don't know which passage you are refering to, but the passage I have
often seen cited as an example of a mother image of God is Isaiah 49:15
"Can a woman forget her sucking child / that she should have no 
compassion / on the son of her womb? / Even these may forget, / 
yet I will not forget you." 
 
This passage is *not* a mother image of God at all. The mother here
is the image of the best human constancy can show, and it is 
contrasted with the constancy of God. The mother figure here represents
mankind, not God.
-- 
Mark Baker                  | "The task ... is not to cut down jungles, but 
aa888@Freenet.carleton.ca   | to irrigate deserts." -- C. S. Lewis

[Luke 13:34   O Jerusalem, Jerusalem, killing the prophets and stoning those
who are sent to you! How often would I have gathered your children together
as a hen gathers her brood

From the above it is quite clear the the entries correspond to politics, religion and graphics respectively. Let's see if this is correct?

In [32]:
# obtain index of the target labels
newsgroups.target[:3]

array([0, 5, 1])

In [33]:
# obtain the corresponding elements from target_names
np.array(newsgroups.target_names)[newsgroups.target[:3]]

array(['alt.atheism', 'soc.religion.christian', 'comp.graphics'],
      dtype='<U22')

Notice that the first one has be categorized under atheism while we had guessed politics! This shows the intersectionality between atheism and politics!

In [233]:
# number of documents in each category
for ind in range(7):
    target = newsgroups.target_names[ind]
    print('{}: {} '.format(target,(newsgroups.target == ind).astype(int).sum()))

alt.atheism: 480 
comp.graphics: 584 
misc.forsale: 585 
rec.autos: 594 
sci.crypt: 595 
soc.religion.christian: 599 
talk.politics.guns: 546 


## Data Preprocessing

For puposes of topic modeling, we will first preprocess our data. We will therefore reduce all the words to lower case, remove stop words and apply stemming/lemmatization.

In [77]:
from nltk import stem # This contains the PorterStemmer as well as the WordNetLemmatizer of nltk
from nltk.corpus import stopwords

In [35]:
porter = stem.PorterStemmer()

In [51]:
# the preprocessing function will stem all the words and convert them to lowercase
preproc= lambda string: ' '.join(map(lambda word: porter.stem(word), string.lower().split()))

In [52]:
newsgroups.data[0]

'\n\nFind an encyclopedia. Volume H. Now look up Hitler, Adolf. He had\nmany more people than just Germans enamoured with him.\n\nP.'

In [53]:
preproc(newsgroups.data[0])

'find an encyclopedia. volum h. now look up hitler, adolf. he had mani more peopl than just german enamour with him. p.'

In [54]:
from sklearn.feature_extraction.text import CountVectorizer

In [99]:
# after stemming the documents, some of the words may no longer be identified as stop-words as their stemmed version 
# will not be in the list of stop words. These words will therefore not be removed from the text
# to cure this, apply stemming to the stop words and add them to the list of stop words
# for e.g. see joeln's reply in this post: 
# https://stackoverflow.com/questions/57340142/user-warning-your-stop-words-may-be-inconsistent-with-your-preprocessing
stop_words = set(stopwords.words('english'))
print('no. of stop words before stemming: {}'.format(len(stop_words)))
stemmed_stop_words = set(map(lambda word: preproc(word), stop_words))
stop_words.update(stemmed_stop_words)
print('no. of stop words after stemming: {}'.format(len(stop_words)))

no. of stop words before stemming: 179
no. of stop words after stemming: 201


In [108]:
# create a vectorizer with english stop words and the above preprocessing function
# the vectorizer automatically ignores all punctuations
vectorizer = CountVectorizer(stop_words = stop_words, preprocessor = preproc )

In [122]:
# fit the vectorizer to the data to obtain a bag-of-words matrix
# To test the our preprocessing function, let's just try to fit the vectorizer only on the first document
bow = vectorizer.fit_transform(newsgroups.data[0:1]).todense()
bow.shape

(1, 10)

In [121]:
# the individual tokens can be obtained from the get_feature_names() attribute of the vectorizer
vectorizer.get_feature_names()

['adolf',
 'enamour',
 'encyclopedia',
 'find',
 'german',
 'hitler',
 'look',
 'mani',
 'peopl',
 'volum']

The feature names produced by our vectorizer seem to match with the 1st document in our corpora. The preprocessing function that we created above seems to be working fine. Let's now fit the vectorizer to the entire dataset.

In [123]:
bow = vectorizer.fit_transform(newsgroups.data).todense()
print('shape of document-term matrix is: {}'.format(bow.shape))
# print some of the tokens generated by the vectorizer
# The first few ones seem to be numbers and emails etc. so I chose the indices to correspond to english words
print('Sample of tokens generated: {}'.format(vectorizer.get_feature_names()[7500:7510]))

shape of document-term matrix is: (3983, 38819)
Sample of tokens generated: ['bells', 'belong', 'belonging', 'belongs', 'belorusskaya', 'belov', 'beloved', 'belt', 'belts', 'beltway']


In [210]:
vocab = np.array(vectorizer.get_feature_names())
vocab.shape

(38819,)

## SVD: Singular Value Decomposition

Let us now apply SVD for topic modeling. Also, we will use sklearn's [```randomized_svd``` ](https://scikit-learn.org/stable/modules/generated/sklearn.utils.extmath.randomized_svd.html) since it is much faster

In [213]:
from sklearn.utils.extmath import randomized_svd

In [255]:
num_topics = 20
U, S, V = randomized_svd(bow, n_components = num_topics)

In [256]:
print('shape of U: {}'.format(U.shape))
print('shape of S: {}'.format(S.shape))
print('shape of V: {}'.format(V.shape))

shape of U: (3983, 20)
shape of S: (20,)
shape of V: (20, 38819)


In [257]:
S[:10]

array([613.60405179, 574.17273772, 362.07868416, 344.71683751,
       299.56507465, 255.67491512, 237.53532097, 212.53588514,
       194.21425178, 180.92764131])

Let us now display the top 10 words in each of the topics

In [258]:
top_k = 10
top_k_word_indices = V.argsort(axis = 1)[:, :-top_k-1:-1]

In [259]:
def show_topic_words(topic_vector, top_k = 10):
    top_word_indices = topic_vector.argsort()[:-top_k-1:-1]
    words = vocab[top_word_indices]
    weights = topic_vector[top_word_indices]
    # zip the weights and words and make a string our of them
    topic = '+'.join(map(lambda tup: str(tup[0])[:5]+'*'+tup[1] ,zip(weights,words)))
    return topic

In [260]:
# let us view the top 10 topics
for idx, vector in enumerate(V[:10]):
    print('topic: {}'.format(idx+1))
    print(show_topic_words(vector))

topic: 1
0.873*db+0.217*mov+0.189*bh+0.142*cs+0.140*si+0.090*bit+0.083*byte+0.078*al+0.074*bl+0.074*file
topic: 2
0.378*file+0.239*use+0.204*imag+0.188*jpeg+0.154*edu+0.134*anonym+0.127*pub+0.115*ftp+0.112*mail+0.112*system
topic: 3
0.642*file+0.288*gun+0.139*congress+0.128*firearm+0.124*control+0.119*bill+0.110*state+0.102*mr+0.096*handgun+0.094*rkba
topic: 4
0.495*jpeg+0.339*imag+0.212*gif+0.167*color+0.139*format+0.101*version+0.085*display+0.084*convert+0.081*file+0.081*bit
topic: 5
0.208*edu+0.203*pub+0.136*mail+0.134*ftp+0.099*comput+0.097*data+0.089*internet+0.087*send+0.085*graphic+0.084*file
topic: 6
0.217*00+0.194*edu+0.163*god+0.110*data+0.109*graphic+0.099*128+0.098*3d+0.090*50+0.090*ray+0.085*40
topic: 7
0.660*00+0.245*50+0.211*appears+0.210*40+0.170*10+0.167*art+0.129*25+0.123*20+0.118*1st+0.112*80
topic: 8
0.267*anonym+0.171*post+0.149*edu+0.102*god+0.099*server+0.089*file+0.087*anon+0.083*jpeg+0.080*servic+0.073*mail
topic: 9
0.292*pub+0.232*eff+0.197*god+0.150*jpeg+0.1

From above, we see that topic 1, 2, 4, 5 and 8 seem to correspond to computer science and graphics related subjects. Similarly, topic 3 is clearly related to politics and guns. Topic 6 has a high component of 'god' but other words seem to be cs/graphics related. It is therefore much harder to tell what exactly does it represent. topic 7 mostly seems to be made up of number wich don't seem to tell us much either. topic 9 seems to an admixture of religion, atheism, politics and cs/graphics. Topic 10 can be clearly associated with atheism and religion. 

Thus we see that while some of the topics are very interpretable but still others are hard to make sense of. We also notice that our topics are largely dominated by words related to cs/graphics. I wonder why?

Let's see if using a tf-idf vectorizer instead of count vectorizer makes a difference and gives a better selection of topics

## SVD with tf-idf

In [261]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [262]:
vectorizer = TfidfVectorizer(stop_words = stop_words, preprocessor = preproc)

In [263]:
tfidf_mat = vectorizer.fit_transform(newsgroups.data).todense()
tfidf_mat.shape

/home/prarit/anaconda3/envs/NLP/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['becau'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


(3983, 38819)

In [264]:
num_topics = 20
U,S, V = randomized_svd(tfidf_mat, n_components = num_topics)

In [265]:
print('shape of U: {}'.format(U.shape))
print('shape of S: {}'.format(S.shape))
print('shape of V: {}'.format(V.shape))

shape of U: (3983, 20)
shape of S: (20,)
shape of V: (20, 38819)


In [266]:
S[:10]

array([8.51841333, 4.92967855, 4.37497754, 3.83602063, 3.51430331,
       3.3639679 , 3.04066331, 3.01200984, 2.9947416 , 2.94790133])

In [267]:
# let us view the top 10 topics
for idx, vector in enumerate(V[:10]):
    print('topic: {}'.format(idx+1))
    print(show_topic_words(vector))

topic: 1
0.188*would+0.163*one+0.157*god+0.146*use+0.126*peopl+0.122*key+0.121*like+0.119*think+0.118*know+0.112*get
topic: 2
0.388*key+0.229*chip+0.206*encrypt+0.143*clipper+0.139*use+0.127*escrow+0.109*system+0.109*secur+0.104*bit+0.092*algorithm
topic: 3
0.330*key+0.199*god+0.184*chip+0.175*encrypt+0.124*clipper+0.116*escrow+0.088*secur+0.083*law+0.083*govern+0.076*believ
topic: 4
0.341*gun+0.178*car+0.116*right+0.101*peopl+0.099*crime+0.098*weapon+0.090*would+0.086*state+0.085*govern+0.085*law
topic: 5
0.337*00+0.289*car+0.178*god+0.143*key+0.130*price+0.115*sale+0.113*new+0.108*offer+0.104*drive+0.087*chip
topic: 6
0.352*00+0.238*gun+0.143*law+0.100*sale+0.084*control+0.083*offer+0.081*govern+0.081*weapon+0.078*crime+0.078*includ
topic: 7
0.240*00+0.200*could+0.180*ico+0.178*tek+0.176*bobbe+0.174*beauchain+0.174*sank+0.174*bronx+0.173*manhattan+0.168*stay
topic: 8
0.296*gun+0.253*god+0.174*key+0.146*file+0.146*bit+0.141*ico+0.139*tek+0.138*bobbe+0.137*beauchain+0.137*sank
topic: 9

Using a vectorizer based on tfidf seems to a little better. Previously, using the simple vectorizer, we got a topic (topic 7 in the results from previous section) which was largely based on numbers and hence did not make much sense. But upon using tfidf, we do not get such a topic in our list of top 10 topics. Also the topics no longer seem to be dominated by cs/graphics related words.